In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def setup_nltk():
    """
    Downloads necessary NLTK data if not already present.
    """
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet')
    try:
        nltk.data.find('corpora/omw-1.4')
    except LookupError:
        nltk.download('omw-1.4')
    print("NLTK setup complete.")

def load_and_prepare_data(file_path='IMDB Dataset.csv'):
    """
    Loads data from the IMDB Dataset CSV file and prepares it.
    """
    try:
        # Using the full dataset for better accuracy.
        df = pd.read_csv("C:\\Users\\HP\\Downloads\\IMDB Dataset.csv.zip")
        print(f"Data loaded successfully from {file_path}")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure it's in the same directory.")
        return None

    # Drop any potential duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert sentiment labels to Title Case for consistency
    df['sentiment'] = df['sentiment'].str.title()
    
    print("Data preparation complete.")
    print("\nValue counts for each sentiment:")
    print(df['sentiment'].value_counts())
    
    return df


def preprocess_text(text, lemmatizer, stop_words):
    """
    Cleans and preprocesses a single text string.
    """
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Ensure text is a string and lowercase
    text = str(text).lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    # Lemmatize and remove stop words
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(processed_tokens)

def train_model(df):
    """
    Preprocesses data, trains the model, and returns the trained pipeline and test data.
    """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    print("\nPreprocessing text data... This may take a few minutes for the full dataset.")
    df['processed_review'] = df['review'].apply(lambda text: preprocess_text(text, lemmatizer, stop_words))

    X = df['processed_review']
    y = df['sentiment']

    # Using stratify to maintain the same proportion of sentiments in train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Creating a pipeline with a more powerful MultinomialNB (implements a linear SVM by default)
    model_pipeline = make_pipeline(
        TfidfVectorizer(ngram_range=(1, 2)),
        MultinomialNB(random_state=42)
    )

    print("Training the model...")
    model_pipeline.fit(X_train, y_train)
    print("Model training complete.")
    return model_pipeline, X_test, y_test

def evaluate_model(model_pipeline, X_test, y_test):
    """
    Evaluates the model and prints the results for binary classification.
    """
    y_pred = model_pipeline.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy: {accuracy:.2f}\n")

    print("Classification Report:")
    labels = ['Positive', 'Negative']
    print(classification_report(y_test, y_pred, labels=labels, zero_division=0))

    cm = confusion_matrix(y_test, y_pred, labels=labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels,
                yticklabels=labels)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

def predict_sentiment(model_pipeline, review_text):
    """
    Predicts the sentiment of a new, raw review string.
    """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    processed_text = preprocess_text(review_text, lemmatizer, stop_words)
    prediction = model_pipeline.predict([processed_text])
    prediction_proba = model_pipeline.decision_function([processed_text]) # Use decision_function for SGD
    
    # A simple way to get a confidence-like score from the decision function
    confidence = 1 / (1 + np.exp(-np.abs(prediction_proba[0])))

    print(f'Review: "{review_text}"')
    print(f'Predicted Sentiment: {prediction[0]} (Confidence: {confidence:.2f})')
    print('-'*30)

def main():
    """
    Main function to run the sentiment analysis pipeline.
    """
    # Step 1: Setup NLTK
    setup_nltk()

    # Step 2: Load and prepare data from your CSV
    df = load_and_prepare_data('IMDB Dataset.csv')
    
    if df is None:
        return

    # Step 3: Train the model
    model, X_test, y_test = train_model(df)

    # Step 4: Evaluate the model
    evaluate_model(model, X_test, y_test)

    # Step 5: Interactive Prediction
    print("\n--- Interactive Prediction ---")
    predict_sentiment(model, "This was one of the greatest films I have ever seen, a true masterpiece!")
    predict_sentiment(model, "The movie was okay, but the plot was a bit slow and predictable.")
    predict_sentiment(model, "A complete and utter disaster. I can't believe I wasted my time on this.")

if __name__ == '__main__':
    main()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


NLTK setup complete.
Data loaded successfully from IMDB Dataset.csv
Data preparation complete.

Value counts for each sentiment:
sentiment
Positive    24884
Negative    24698
Name: count, dtype: int64

Preprocessing text data... This may take a few minutes for the full dataset.


TypeError: MultinomialNB.__init__() got an unexpected keyword argument 'random_state'

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def setup_nltk():
    """
    Downloads necessary NLTK data if not already present.
    """
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet')
    try:
        nltk.data.find('corpora/omw-1.4')
    except LookupError:
        nltk.download('omw-1.4')
    print("NLTK setup complete.")

def load_and_prepare_data(file_path='IMDB Dataset.csv'):
    """
    Loads data from the IMDB Dataset CSV file and prepares it.
    """
    try:
        # Using the full dataset for better accuracy.
        df = pd.read_csv("")
        print(f"Data loaded successfully from {file_path}")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure it's in the same directory.")
        return None

    # Drop any potential duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert sentiment labels to Title Case for consistency
    df['sentiment'] = df['sentiment'].str.title()
    
    print("Data preparation complete.")
    print("\nValue counts for each sentiment:")
    print(df['sentiment'].value_counts())
    
    return df


def preprocess_text(text, lemmatizer, stop_words):
    """
    Cleans and preprocesses a single text string.
    """
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Ensure text is a string and lowercase
    text = str(text).lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    # Lemmatize and remove stop words
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(processed_tokens)

def train_model(df):
    """
    Preprocesses data, trains the model, and returns the trained pipeline and test data.
    """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    print("\nPreprocessing text data... This may take a few minutes for the full dataset.")
    df['processed_review'] = df['review'].apply(lambda text: preprocess_text(text, lemmatizer, stop_words))

    X = df['processed_review']
    y = df['sentiment']

    # Using stratify to maintain the same proportion of sentiments in train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Creating a pipeline with MultinomialNB as requested
    model_pipeline = make_pipeline(
        TfidfVectorizer(ngram_range=(1, 2)),
        MultinomialNB()
    )

    print("Training the model...")
    model_pipeline.fit(X_train, y_train)
    print("Model training complete.")
    return model_pipeline, X_test, y_test

def evaluate_model(model_pipeline, X_test, y_test):
    """
    Evaluates the model and prints the results for binary classification.
    """
    y_pred = model_pipeline.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy: {accuracy:.2f}\n")

    print("Classification Report:")
    labels = ['Positive', 'Negative']
    print(classification_report(y_test, y_pred, labels=labels, zero_division=0))

    cm = confusion_matrix(y_test, y_pred, labels=labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels,
                yticklabels=labels)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

def predict_sentiment(model_pipeline, review_text):
    """
    Predicts the sentiment of a new, raw review string.
    """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    processed_text = preprocess_text(review_text, lemmatizer, stop_words)
    prediction = model_pipeline.predict([processed_text])
    prediction_proba = model_pipeline.predict_proba([processed_text])
    
    # Get the confidence score for the predicted class
    confidence = np.max(prediction_proba)

    print(f'Review: "{review_text}"')
    print(f'Predicted Sentiment: {prediction[0]} (Confidence: {confidence:.2f})')
    print('-'*30)

def main():
    """
    Main function to run the sentiment analysis pipeline.
    """
    # Step 1: Setup NLTK
    setup_nltk()

    # Step 2: Load and prepare data from your CSV
    df = load_and_prepare_data('IMDB Dataset.csv')
    
    if df is None:
        return

    # Step 3: Train the model
    model, X_test, y_test = train_model(df)

    # Step 4: Evaluate the model
    evaluate_model(model, X_test, y_test)

    # Step 5: Interactive Prediction
    print("\n--- Interactive Prediction ---")
    predict_sentiment(model, "This was one of the greatest films I have ever seen, a true masterpiece!")
    predict_sentiment(model, "The movie was okay, but the plot was a bit slow and predictable.")
    predict_sentiment(model, "A complete and utter disaster. I can't believe I wasted my time on this.")

if __name__ == '__main__':
    main()